# Portfolio Optimization Validation Notebook

This notebook validates the Portfolio Optimization implementation in voiage by demonstrating its functionality with practical examples.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from voiage.methods.portfolio import portfolio_voi
from voiage.schema import DecisionOption, PortfolioSpec, PortfolioStudy, TrialDesign

## Define a Study Value Calculator

First, let's create a function that calculates the value (EVSI) for a given study design.

In [ ]:
def calculate_study_evsi(study: PortfolioStudy) -> float:
    """Calculate EVSI for a study based on its design.
    
    This is a simplified example. In practice, this would involve
    running a full EVSI calculation using the voiage methods.
    """
    # Simple model: EVSI is proportional to sample size and treatment effect
    total_sample_size = study.design.total_sample_size
    
    # Assume treatment effect is related to the number of arms
    n_arms = len(study.design.arms)
    
    # Base value calculation
    base_value = total_sample_size * n_arms * 100
    
    # Adjust based on study name for variety
    if "Pivotal" in study.name:
        return base_value * 2.0
    elif "Exploratory" in study.name:
        return base_value * 0.5
    else:
        return base_value

## Create Sample Portfolio Studies

Let's define a set of candidate research studies with different designs and costs.

In [ ]:
# Define trial designs
design1 = TrialDesign([
    DecisionOption("Standard of Care", 100),
    DecisionOption("New Treatment A", 100)
])

design2 = TrialDesign([
    DecisionOption("Standard of Care", 150),
    DecisionOption("New Treatment B", 150)
])

design3 = TrialDesign([
    DecisionOption("Standard of Care", 200),
    DecisionOption("New Treatment C", 100),
    DecisionOption("New Treatment D", 100)
])

design4 = TrialDesign([
    DecisionOption("Treatment A", 300),
    DecisionOption("Treatment B", 150),
    DecisionOption("Treatment C", 150)
])

design5 = TrialDesign([
    DecisionOption("Standard of Care", 500),
    DecisionOption("New Treatment E", 500)
])

# Create portfolio studies
studies = [
    PortfolioStudy("Exploratory Study 1", design1, 50000),
    PortfolioStudy("Exploratory Study 2", design2, 75000),
    PortfolioStudy("Phase II Study", design3, 150000),
    PortfolioStudy("Pivotal Study", design4, 500000),
    PortfolioStudy("Large Pivotal Study", design5, 1000000)
]

print("Portfolio Studies:")
for i, study in enumerate(studies):
    evsi = calculate_study_evsi(study)
    ratio = evsi / study.cost
    print(f"{i+1}. {study.name}")
    print(f"   Sample size: {study.design.total_sample_size}")
    print(f"   Cost: ${study.cost:,.0f}")
    print(f"   EVSI: ${evsi:,.0f}")
    print(f"   Value/Cost Ratio: {ratio:.6f}")
    print()

## Portfolio Optimization Without Budget Constraint

First, let's optimize the portfolio without any budget constraints.

In [ ]:
# Create portfolio specification without budget constraint
portfolio_no_budget = PortfolioSpec(studies=studies, budget_constraint=None)

# Optimize using greedy algorithm
result_no_budget = portfolio_voi(
    portfolio_specification=portfolio_no_budget,
    study_value_calculator=calculate_study_evsi,
    optimization_method="greedy"
)

print("Optimization Results (No Budget Constraint):")
print(f"Selected Studies: {[s.name for s in result_no_budget['selected_studies']]}")
print(f"Total Value: ${result_no_budget['total_value']:,.0f}")
print(f"Total Cost: ${result_no_budget['total_cost']:,.0f}")
print(f"Details: {result_no_budget['method_details']}")

## Portfolio Optimization With Budget Constraint

Now let's optimize with a budget constraint of $750,000.

In [ ]:
# Create portfolio specification with budget constraint
budget = 750000
portfolio_with_budget = PortfolioSpec(studies=studies, budget_constraint=budget)

# Optimize using greedy algorithm
result_greedy = portfolio_voi(
    portfolio_specification=portfolio_with_budget,
    study_value_calculator=calculate_study_evsi,
    optimization_method="greedy"
)

print(f"Optimization Results (Budget: ${budget:,.0f}):")
print(f"Selected Studies: {[s.name for s in result_greedy['selected_studies']]}")
print(f"Total Value: ${result_greedy['total_value']:,.0f}")
print(f"Total Cost: ${result_greedy['total_cost']:,.0f}")
print(f"Remaining Budget: ${budget - result_greedy['total_cost']:,.0f}")
print(f"Details: {result_greedy['method_details']}")

## Comparison of Optimization Methods

Let's compare the results from different optimization methods.

In [ ]:
# Compare greedy and integer programming methods
result_ip = portfolio_voi(
    portfolio_specification=portfolio_with_budget,
    study_value_calculator=calculate_study_evsi,
    optimization_method="integer_programming"
)

print("Comparison of Optimization Methods:")
print(f"Greedy Method:")
print(f"  Selected Studies: {[s.name for s in result_greedy['selected_studies']]}")
print(f"  Total Value: ${result_greedy['total_value']:,.0f}")
print(f"  Total Cost: ${result_greedy['total_cost']:,.0f}")
print()
print(f"Integer Programming Method:")
print(f"  Selected Studies: {[s.name for s in result_ip['selected_studies']]}")
print(f"  Total Value: ${result_ip['total_value']:,.0f}")
print(f"  Total Cost: ${result_ip['total_cost']:,.0f}")

## Sensitivity Analysis

Let's perform a sensitivity analysis by varying the budget constraint.

In [ ]:
# Vary budget and calculate optimal portfolio value
budgets = [100000, 200000, 300000, 500000, 750000, 1000000, 1500000, 2000000]
greedy_values = []
ip_values = []

for budget in budgets:
    portfolio_spec = PortfolioSpec(studies=studies, budget_constraint=budget)
    
    # Greedy method
    result_greedy = portfolio_voi(
        portfolio_specification=portfolio_spec,
        study_value_calculator=calculate_study_evsi,
        optimization_method="greedy"
    )
    greedy_values.append(result_greedy['total_value'])
    
    # Integer programming method
    result_ip = portfolio_voi(
        portfolio_specification=portfolio_spec,
        study_value_calculator=calculate_study_evsi,
        optimization_method="integer_programming"
    )
    ip_values.append(result_ip['total_value'])

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(budgets, greedy_values, 'bo-', label='Greedy Method', linewidth=2, markersize=8)
plt.plot(budgets, ip_values, 'ro-', label='Integer Programming', linewidth=2, markersize=8)
plt.xlabel('Budget Constraint ($)')
plt.ylabel('Total Portfolio Value ($)')
plt.title('Portfolio Value vs Budget Constraint')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xscale('log')
plt.yscale('log')
plt.show()

# Print numerical results
print("Budget Sensitivity Analysis:")
print("Budget\t\tGreedy Value\tIP Value\tDifference")
print("-" * 50)
for budget, greedy_val, ip_val in zip(budgets, greedy_values, ip_values):
    diff = ip_val - greedy_val
    print(f"${budget:,}\t\t${greedy_val:,.0f}\t\t${ip_val:,.0f}\t\t${diff:,.0f}")

## Summary

This notebook has demonstrated:

1. How to set up portfolio optimization problems with voiage
2. How to define study value calculators
3. How to perform portfolio optimization with and without budget constraints
4. How to compare different optimization methods
5. How to perform sensitivity analysis on budget constraints

The voiage portfolio optimization implementation provides a flexible framework for research prioritization and resource allocation in health economics and other domains.